In [2]:
import sys
sys.path.insert(0, 'D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\Classes')

import warnings
warnings.filterwarnings('ignore')


from models import ClassificationModel, EmbeddingsModel, SiameseModel
from trainer import SiameseTrainerThreeClass, all_subjects, all_subjects_intense

import numpy as np
from tqdm import tqdm
import csv
import torch
import os
from os import listdir
from os.path import isfile, join


In [3]:
#get all models
path = "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\ThreeClassSiamese\pretrained_models\\"
all_files = [f for f in listdir(path) if isfile(join(path, f))]

# LOSO - INTENSE

In [5]:
#load each model
for file in all_files:
    model_path = path+file
    saving_path = "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\ThreeClassSiamese\\results\\no_finetuning\\"+file[:-4]+".csv"
    model_path = "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\ThreeClassSiamese\pretrained_models\\"+file

    #generate file for LOSO results
    with open(saving_path, mode='a') as result_file:
        results_writer = csv.writer(result_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        results_writer.writerow(["subj", "accuracy", "cm"])

    #LOSO for testing
    for subj in tqdm(all_subjects_intense):
        subjects_test = [subj]
        subjects_train = [sub for sub in all_subjects_intense if sub not in subjects_test]

        trainer_hyperparameter = {
            "path": "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\FeatureGeneration\dataset_processed\INTENSE2\\normalized_subjects.pkl",
            "subjects_train": subjects_train,
            "subjects_test": subjects_test,
            "intense_dataset_train": True,
            "intense_dataset_test": True,
            "xite_2class_train": False,
            "xite_2class_test": False,
            "indices1_train": 4,
            "indices2_train": 1,
            "indices1_test": 1,
            "indices2_test": 0,
            "use_regression": False,
            "wandb": False,
            "log": False,
            "learning_rate": 0.001,
            "batch_size": 256,
            "batch_size_test": 256,
            "freeze_embed": False,
            "decision_function": 3,
            "filter": [],
            "weight_decay": 0,
            "lr_steps": 10,
            "number_steps": 400,
            "number_steps_testing": None
        }

        model_embedding_hyperparameter = {
                "sigmoid": False,
                "dropout": 0.35,
                "layers": [207, 512, 64]
        }

        if trainer_hyperparameter["use_regression"]:
            head = 1
        else:
            head=2

        model_classifier_hyperparameter = {
            "dropout": 0.35,
            "layers": [64, 32],
            "head_type": head
        }

        classifier_model = ClassificationModel(model_classifier_hyperparameter)
        embedding_model = EmbeddingsModel(model_embedding_hyperparameter)
        siamese_model = SiameseModel(embedding_model, classifier_model, decision_function=trainer_hyperparameter["decision_function"])
        
        _ = siamese_model.load_state_dict(torch.load(model_path))
        
        trainer = SiameseTrainerThreeClass(trainer_hyperparameter, siamese_model, device="cuda:0")

        #test network and get results
        res = trainer.test()
        cm = res["cm"]
        acc = res["acc"]

        #normalice cm
        s = np.sum(cm, axis=1)
        cm = cm.astype('float64')
        cm[0] = cm[0]/s[0]
        cm[1] = cm[1]/s[1]
        cm[2] = cm[2]/s[2]

        with open(saving_path, mode='a') as result_file:
            results_writer = csv.writer(result_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            results_writer.writerow([subj, acc*100, cm])

100%|██████████| 24/24 [00:01<00:00, 13.42it/s]
